In [ ]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 155 (delta 18), reused 1 (delta 1), pack-reused 114
Receiving objects: 100% (155/155), 122.48 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [ ]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
#from MLDL2024_semantic_segmentation.datasets.importDataset import Download
#from MLDL2024_semantic_segmentation.datasets.importDataset import Modified_CityScapes
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.models.deeplabv2.deeplabv2 import *
from MLDL2024_semantic_segmentation.train import * 
from MLDL2024_semantic_segmentation.models.metrics import *

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Setup fixed parameters
num_epochs = 5
num_classes = 19

In [ ]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
])

# Create dataloader
dataset_train = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', split = 'train', transform = transform_image, label_transform = transform_target)
dataloader_train = DataLoader(dataset_train, batch_size=16, shuffle=True)

dataset_val = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', split = 'val', transform = transform_image, label_transform = transform_target)
dataloader_val = DataLoader(dataset_train, batch_size=16, shuffle=False)

In [ ]:

# Pretrained
pretrained = torchvision.models.resnet101(pretrained=True).to(device)
torch.save(pretrained.state_dict(), "DeepLab_resnet_pretrained_imagenet.pth")

# Inizialization of the model
model = get_deeplab_v2().to(device)

#Putting on the 2 gpus
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)




In [ ]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

mIOU = 0

for epoch in range(num_epochs):
    train(model, optimizer, dataloader_train, loss_fn)
    mIOU = test(model, dataloader_val, loss_fn)
    print(f"epoch: {epoch}, Validation IOU: {mIOU:.2f}")    

    '''
    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'val_IOU': val_IOU
    },"checkpoint.pth.tar")
    '''
print(f"Final mIOU: {mIOU:.2f}")    

flops = Flops(model, 1024, 512)

print(f"Number of flops?: {flops}")

latency = Latency_FPS(model, 1024, 512)

print(f"Latency: {latency}")

print(f"number of parameters: {model.count_params()}")